# Diferenciación automática (o algorítmica): Parte 1

### NOTA

Este notebook se iniciará en la clase y **debe** hacerse en equipo de dos personas (máximo tres). La resolución completa de los ejercicios debe ser enviada como "Tarea4.ipynb".

## Motivación

En el [notebook anterior](https://github.com/lbenet/2016-2_TSFisicaComputacional/blob/master/notas_clase/06_Derivacion_numerica.ipynb), vimos que hay diferentes formas de implementar la derivación de una función $f(x)$ en un punto $x_0$ y que éstas dependen de un parámetro $h$, que es la separación entre puntos cercanos. 

Vimos que el error absoluto en términos de $h$, cuando $h\to 0$, tiene un comportamiento distinto, y que de hecho puede ser "contaminado" por errores de numéricos.

Se obtuvo que:

- La "derivada derecha" converge *linealmente* al $h\to 0$ al valor correcto de la derivada, es decir, el error es proporcional a $\mathcal{O}(h)$. Sin embargo, para valores *suficientemente* pequeños de $h$, el valor obtenido de la derivada deja de tener sentido ya que pierde exactitud.

- La "derivada simétrica" converge *cuadráticamente*, es decir, el error escala como $\mathcal{O}(h^2)$. Al igual que la derivada derecha, para $h$ suficientemente pequeña errores "de cancelación" debidos a la diferencia en la definición hacen que el resultado pierda sentido.

- La definición de la "derivada compleja" también converge *cuadráticamente*; a diferencia de las dos definiciones anteriores *no* exhibe problemas al considerar valores de $h$ muy pequeños.

Los puntos anteriores muestran que, la manera de implementar un algoritmo (concreto) cualquiera en la computadora es importante respecto a cuestiones de la convergencia y del manejo de errores numéricos. En este sentido, la "derivada compleja" da el resultado que más se acerca al exacto, incluso para valores de $h \sim 10.^{-16}$.

La pregunta es si podemos ir más allá y obtener el valor exacto, esto es, el que más se acerca al valor obtenido con números reales excepto por cuestiones de redondeo.

## Diferenciación automática (o algorítmica)

Citando a [wikipedia](https://en.wikipedia.org/wiki/Automatic_differentiation): ``Automatic differentiation (AD), also called algorithmic differentiation or computational differentiation, is a set of techniques to numerically evaluate the derivative of a function specified by a computer program.''

Diferenciación automática **no es** diferenciación numérica. Está basada en cálculos numéricos (evaluación de funciones por la computadora), pero **no** usa ninguna de las definiciones por diferencias finitas, como las que vimos la clase anterior.

Para ilustrar su funcionamiento, empezaremos recordando cómo funcionan los números complejos: $z = a + \mathrm{i} b$.

Uno puede definir todas las operaciones aritméticas de la manera natural (a partir de los números reales), manteniendo las expresiones con $\mathrm{i}$ factorizada; en el caso de la multiplicación y división, simplemente recordamos que $\mathrm{i}^2=-1$.

Por ejemplo, tenemos que, para $z = a + \mathrm{i} b$ y $w = c + \mathrm{i} d$
\begin{eqnarray*}
z + w & = & (a + \mathrm{i} b) + (c + \mathrm{i} d) = a + c + \mathrm{i}(b + d),\\
z \cdot w & = & (a + \mathrm{i} b)\cdot (c + \mathrm{i} d) \\
  & = & ac + \mathrm{i} (ad+bc) + \mathrm{i}^2 b d
 = ac - b d + \mathrm{i} (ad+bc).
\end{eqnarray*}

Por último, vale también la pena recordar que, $\mathbb{C}$ es *isomorfo* a $\mathbb{R}^2$, esto es, hay un mapeo biyectivo de $ z \leftrightarrow (a, b)$.

Volviendo a la cuestión de la diferenciación automática, la idea es introducir un par ordenado donde la primer
componente es el valor de una función $f(x)$ evaluada en $x_0$, y la segunda es el valor de su derivada evaluada
en el mismo punto. Esto es, definimos a los *duales* como:

\begin{equation}
\vec{f}(x_0) = \big( f_0, f'_0\big) = f_0 + \hat{\mathbf{h}}\, f'_0,
\end{equation}

donde $f_0 = f(x_0)$ y $f'_0=\frac{d f}{d x}(x_0)$ y, en la segunda igualdad, $\hat{\mathbf{h}}$ sirve para indicar la segunda componente del par ordenado. (En un sentido que se precisará más adelante, $\hat{\mathbf{h}}$ se comporta de una manera semejante a $\mathrm{i}$ para los números complejos, distinguiéndose en $\hat{\mathbf{h}}^2$.)

En particular, para la función constante $f(x)=c$ se cumple que el dual asociado es $\vec{c}(x_0)=(c,0)$, y para la función identidad $f(x)=x$ obtenemos $\vec{x}(x_0)=(x_0,1)$. Vale la pena aquí enfatizar que la función identidad es precisamente la variable independiente, respecto a la que se está derivando.

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina los duales, donde el parámetro debe ser un subtipo de `Real` (ver la siguiente celda). La parte que identifica a $f_0$ será llamada `fun`, y la correspondiente a $f'_0$ será `der`.

La definición debe incluir métodos que sean compatibles con las dos propiedades arriba mencionadas, es decir, que el dual de una constante (cualquier número real) sea $(c,0)$, y que el de la variable independiente sea $(x_0,1)$. Para lo segundo definiremos una función `xdual` con la propiedad mencionada.

In [2]:
"""Definición de los duales, donde
...
"""
type Dual
    fun::Real
    der::Real
end
Dual(a::Real) = Dual(a,0)

Dual

In [3]:
#Test
Dual(15.1,3)

Dual(15.1,3)

In [4]:
# Definan un método que permita la promoción automática cuando no son del mismo tipo
import Base: convert, promote_rule
convert(::Type{Dual},x::Real) = Dual(x,0)
promote_rule(::Type{Dual}, ::Type{Int} ) = Dual
promote_rule(::Type{Dual}, ::Type{Float64} ) = Dual
Dual(a::Irrational,b::Irrational) = Dual(convert(Float64,a),convert(Float64,b))
Dual(a::Irrational,b::Real) = Dual(convert(Float64,a),convert(Float64,b))
Dual(a::Real,b::Irrational) = Dual(convert(Float64,a),convert(Float64,b))

Dual

In [5]:
# Aqui se define la función `xdual`, que se usará para identificar la variable independiente
# La función dpende de x_0, y debe regresar el Dual apropiado a la variable independiente
xdual(x0)=Dual(x0,1)

xdual (generic function with 1 method)

In [6]:
#Test
xdual(1000)

Dual(1000,1)

## Aritmética de duales

De la definición, y recordando el significado de cada una de las componentes del par doble, tenemos que las operaciones aritméticas quedan definidas por:

\begin{eqnarray}
    \vec{u} \pm \vec{w} &=& \big( u_0 \pm w_0,\, u'_0\pm w'_0 \big),\\
    \vec{u} \times \vec{w} &=& \big( u_0 \cdot w_0,\, u_0 w'_0 +  w_0 u'_0 \big),\\
    \frac{\vec{u}}{\vec{w}} &=& \big( \frac{u_0}{w_0},\, \frac{ u'_0 - (u_0/w_0)w'_0}{w_0}\big),\\
    {\vec{u}}^\alpha &=& \big( u_0^\alpha,\, \alpha u_0^{\alpha-1} u'_0 \big).\\
\end{eqnarray}    

Además, están los operadores unitarios, que satisfacen:
\begin{equation}
    \pm \vec{u} = \big(\pm u_0, \pm u'_0 \big).
\end{equation}    

Noten que las dos relaciones anteriores son compatibles con $\hat{\mathbf{h}}^2=0$.

### Ejercicio

Implementen *todas* las operaciones aritméticas. Estas operaciones deben incluir las operaciones aritméticas que involucran un número cualquiera (`a :: Real`) y un dual (`b::Dual`). Esto se puede hacer implementando los métodos específicos para estos casos (¡y que sirven en cualquier órden!). Implementen también la comparación entre duales (`==`). Incluyan tests que muestren que cada una de ellas está bien definida, y que sus resultados dan valores consistentes.

In [9]:
# Aqui se implementan los métodos necesarios para cada función

import Base: +, -, *, /, ^, == 
+(x::Dual,y::Dual) =Dual( x.fun+y.fun , x.der+y.der )
-(x::Dual,y::Dual) =Dual( x.fun-y.fun , x.der-y.der )
*(x::Dual,y::Dual) =Dual( x.fun*y.fun , (x.fun*y.der)+(y.fun*x.der) )
/(x::Dual,y::Dual) =Dual( x.fun/y.fun , (x.der-((x.fun/y.fun)*y.der))/y.fun )

+(x::Dual,y::Real) =Dual( x.fun+y , x.der )
-(x::Dual,y::Real) =Dual( x.fun-y , x.der )
*(x::Dual,y::Real) =x*convert(Dual,y)
/(x::Dual,y::Real) =x/convert(Dual,y)

+(y::Real,x::Dual) =Dual( x.fun+y , x.der )
-(y::Real,x::Dual) =Dual( x.fun-y , x.der )
*(y::Real,x::Dual) =x*convert(Dual,y)
/(y::Real,x::Dual) =x/convert(Dual,y)

^(x::Dual,a::Integer) =Dual( x.fun^a , a*(x.fun^(a-1))*x.der )
^(x::Dual,a::Real) =Dual( x.fun^a , a*(x.fun^(a-1))*x.der )

+(x::Dual) =Dual( +x.fun , +x.der )
-(x::Dual) =Dual( -x.fun , -x.der )

==(x::Dual,y::Dual)=(x.fun,x.der)==(y.fun,y.der)

import Base: print,show
show(io::IO,x::Dual)=print(io, x.fun," + ",x.der,"ε")
print(io::IO,x::Dual)=show(io,x)

print (generic function with 19 methods)

In [8]:
#Test
a=Dual(15,3)
b=Dual(2,5)
c=2
print("a: ",a,"\n")
print("b: ",b,"\n")
print("a+b: ",a+b,"\n")
print("a-b: ",a-b,"\n")
print("a*b: ",a*b,"\n")
print("a/b: ",a/b,"\n")
print("a^2: ",a^2,"\n")
print("a^.5: ",a^.5,"\n")
print("a==b: ",a==b,"\n")
print("a==a: ",a==a,"\n")

a: 15 + 3ε
b: 2 + 5ε
a+b: 17 + 8ε
a-b: 13 + -2ε
a*b: 30 + 81ε
a/b: 7.5 + -17.25ε
a^2: 225 + 90ε
a^.5: 3.872983346207417 + 0.38729833462074165ε
a==b: false
a==a: true


In [10]:
# Aqui se incluyen las pruebas necesarias
using Base.Test
A=(0,1,7,0.0,1.0,7.0,pi,-1,-7)
for (a in A)
    for(b in A)
        d=Dual(a,b);
        c=xdual(a)
        if(a!=0)
            @test (d+d-d*d/d)^1==d
            @test (d+a-a*a/a)^1==d
            @test (c+d-d*d/d)^1==c
        else
            @test ((d+d-d)*d)^1==d*d
            @test ((d+a-a)*a)^1==d*a
            @test ((c+d-d)*d)^1==c*d
        end
    end
end

Ahora probaremos la infraestructura que han desarrollado hasta ahora. En particular, usaremos la misma función que se
utilizón en la clase anterior:

\begin{equation}
    f_\textrm{test}(x) = 3x^3 - 2,
\end{equation}

y la idea es calcular la derivada en $x_0=1$.

In [11]:
f_test(x) = 3*x^3 - 2

f_test (generic function with 1 method)

In [12]:
f_test( xdual(1) )

1 + 9ε

In [15]:
# Del resultado anterior, obtengan el valor de la derivada
f_test( xdual(1) ).der

9

Consideremos otra función racional:

$$
g_\textrm{test}(x) = (3x^2-8x+5)/(7x^3-1)
$$

cuya derivada queremos calcular en $x_0=1$. Según [WolframAlpha](http://www.wolframalpha.com/input/?i=D%5B+%283x%5E2-8x%2B5%29%2F%287x%5E3-1%29+%2C+x+%5D+%2F.+x-%3E+1), el resultado exacto es $-1/3$.


In [13]:
gtest(x) = (3x^2-8x+5)/(7x^3-1)

gtest (generic function with 1 method)

In [14]:
gtest(xdual(1))

0.0 + -0.3333333333333333ε

El resultado anterior es la representación en números de punto flotante de $-1/3$. La pregunta es si podemos obtener
el resultado exacto.

In [20]:
# Evalúen `gtest` de manera que el resultado sea el exacto
gtest(xdual(1//1))

0//1 + -1//3ε

Para entender cómo funciona esto, evaluaremos explícitamente y paso a paso $g_\textrm{test}(x)$ en $\vec{x_0} = (1,1)$:

\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = \frac{\vec{3}\cdot{\vec{x_0}}^2-\vec{8}\cdot\vec{x_0}+\vec{5}}{\vec{7}\cdot{\vec{x_0}}^3-\vec{1}}
\end{equation}


\begin{eqnarray}
\vec{g}_\textrm{test}(\vec{x_0}) & = & 
\frac{(3,0)\cdot{(1,1)}^2-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot{(1,1)}^3-(1,0)}\\
& = & \frac{(3,0)\cdot(1,2)-(8,0)\cdot(1,1)+(5,0)}{(7,0)\cdot(1,3)-(1,0)}
\end{eqnarray}


\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = 
\frac{ (3,6)-(8,1)+(5,0)}{(7,21)-(1,0)} = \frac{ \Big(3-8+5, 6-8\Big)}{\Big( 7-1,21\Big)} = 
\frac{ \Big(0, -2\Big)}{\Big( 6,21\Big)} 
\end{equation}

\begin{equation}
\vec{g}_\textrm{test}(\vec{x_0}) = \Big(\frac{0}{6}, \frac{(-2)-21\cdot(0/6)}{6} \Big) = 
\Big(0, -\frac{1}{3}\Big). \Box
\end{equation}


## Modulos y "runtests.jl" en julia

### Ejercicio

Para reutilizar el código que han hecho en este notebook, y de hecho seguirlo desarrollando, conviene ponerlo dentro de un módulo. Para hacer esto, deberán copiar todo el código necesario (y que aparece en la resolución de los ejercicios anteriores) en un archivo coyo nombre será "AutomDiff.jl" y cuya estructura es:

```julia
# Aqui viene una explicación de lo que se hace en el módulo, los autores y la fecha

# La siguiente instrucción sirve para *precompilar* el módulo
__precompile__(true)

module AD
    import Base: +, -, *, /, ^
    
    export Dual, xdual
    
    # Aqui viene TODO el código que implementaron

end
```

Todas las pruebas deberán ser incluídas en un archivo llamado "runtest.jl", y su estructura es:

```julia
# Este archivo incluye los tests del módulo AD
include("AutomDiff.jl")
using Base.test
using AD

# A continuación vienen los tests que implementaron y que deben 
# ser suficientemente exhaustivos


```

Estos dos archivos deben incluirlos en el envío de su tarea.

In [ ]:
Adjunto los archivos